In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [2]:
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

In [3]:
import numpy as np

# 70で作成した行列をロード
X_train = np.load("./matrix/x_train.npy")
Y_train = np.load("./matrix/y_train.npy")

In [4]:
X_train_tensor = torch.from_numpy(X_train)
Y_train_tensor =torch.from_numpy(Y_train)
datasets = TextDataset(X_train, Y_train)
train_dataloader = DataLoader(datasets, shuffle=True, batch_size=64)

In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(300, 4),
            nn.Softmax()
        )

    def forward(self, x):
        logits = self.layer(x)
        return logits

In [6]:
model = Model()
learning_rate = 1e-3
batch_size = 64
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 予測と損失の計算
        X = X.float()
        pred = model.forward(X)
        loss = loss_fn(pred, y.to(torch.float64))
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
print("Before training!\n ------------------------------")
for name, param in model.named_parameters():
    print(f"{name}: {param.data}")
    print(f"Shape: {param.shape}")
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("Done training!\n ------------------------------")
for name, param in model.named_parameters():
    print(f"{name}: {param.data}")
    print(f"Shape: {param.shape}")

Before training!
 ------------------------------
layer.0.weight: tensor([[-0.0201,  0.0326, -0.0101,  ...,  0.0101, -0.0346, -0.0562],
        [-0.0469,  0.0398, -0.0533,  ..., -0.0082,  0.0521, -0.0107],
        [ 0.0328,  0.0090, -0.0035,  ...,  0.0179, -0.0026, -0.0159],
        [ 0.0365, -0.0018,  0.0335,  ...,  0.0245, -0.0102, -0.0290]])
Shape: torch.Size([4, 300])
layer.0.bias: tensor([-0.0291,  0.0315, -0.0109, -0.0397])
Shape: torch.Size([4])
Epoch 1
-------------------------------
loss: 1.389865  [    0/10671]
loss: 1.387464  [ 6400/10671]
Epoch 2
-------------------------------
loss: 1.385743  [    0/10671]
loss: 1.387825  [ 6400/10671]
Epoch 3
-------------------------------
loss: 1.385952  [    0/10671]


/Users/tajirimanato/.pyenv/versions/3.12.2/lib/python3.12/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 1.385043  [ 6400/10671]
Epoch 4
-------------------------------
loss: 1.381341  [    0/10671]
loss: 1.381144  [ 6400/10671]
Epoch 5
-------------------------------
loss: 1.379035  [    0/10671]
loss: 1.379519  [ 6400/10671]
Epoch 6
-------------------------------
loss: 1.377112  [    0/10671]
loss: 1.375037  [ 6400/10671]
Epoch 7
-------------------------------
loss: 1.374575  [    0/10671]
loss: 1.374786  [ 6400/10671]
Epoch 8
-------------------------------
loss: 1.373525  [    0/10671]
loss: 1.370867  [ 6400/10671]
Epoch 9
-------------------------------
loss: 1.376710  [    0/10671]
loss: 1.370159  [ 6400/10671]
Epoch 10
-------------------------------
loss: 1.368776  [    0/10671]
loss: 1.367580  [ 6400/10671]
Done training!
 ------------------------------
layer.0.weight: tensor([[-0.0214,  0.0347, -0.0113,  ...,  0.0143, -0.0298, -0.0617],
        [-0.0481,  0.0381, -0.0504,  ..., -0.0079,  0.0484, -0.0096],
        [ 0.0376,  0.0107, -0.0085,  ...,  0.0125, -0.0010, -0.013